## Problem 1

In [1]:
from math import *
from scipy.special import factorial, gammaln, erf
from scipy.optimize import fsolve
import numpy as np
import pandas as pd

#### (a) Birthday Paradox 

In [2]:
days = 365
n = 23
fraction = exp(gammaln(days+1) - gammaln(days-n + 1))
fraction *= (1/pow(days, n))

prob = 1 - fraction
print(prob)
percentage = round(prob*100)
print("For {} people the birthday paradox yields {}%".format(n, percentage))

0.5072972343240897
For 23 people the birthday paradox yields 51%


#### (b) fsolve

In [3]:
days = 365
f = lambda n: 0.5 - ((exp(gammaln(days + 1) - gammaln(days - n + 1))) * (1/pow(days, n))) 
x = fsolve(f, x0=23, xtol=pow(10,-10))[0]
print("Solution:",x)

Solution: 22.767690315612928


-----

## Problem 2

In [4]:
def bisect(f, a, b, tol=pow(10, -4)):
    fa = f(a)
    fb = f(b)
    k = 0
    while abs((b-a))/2 > tol:
        c = (a+b)/2
        fc = f(c)
        
        if (fa*fc < 0):
            b = c
            fb = fc
        else:
            a = c
            fa = fc
    return c

In [5]:
def false_position(f, a, b, tol=pow(10, -4)):
    fa = f(a)
    fb = f(b)
    fc = 100
    
    while abs(fc) > tol:
        c = (a*fb-b*fa)/(fb-fa)
        fc = f(c)

        if (fa*fc < 0):
            b = c
            fb = fc
        else:
            a = c
            fa = fc
        #print(c)
        
        
    return c

a) Bisection method

In [6]:
f = lambda x: np.tan(x) - x

p = bisect(f,2,5) 
print("Sol 1:",p)
p = bisect(f, p, p+np.pi)
print("Sol 2:",p)

p = bisect(f, p, p+np.pi)
print("Sol 3:",p)

Sol 1: 4.49334716796875
Sol 2: 7.634748073960058
Sol 3: 10.776148979951365


b) Regula-Falsi

In [7]:
f = lambda x: np.tan(x) - x

p = false_position(f,3.9,4.6) 
print("Sol 1:",p)

p = false_position(f, 5.6, 7.7)
print("Sol 2:",p)

p = false_position(f, 8.15, 10.93)
print("Sol 3:",p)


Sol 1: 4.493405305810955
Sol 2: 7.725250976308113
Sol 3: 10.904120912381885


------

## Problem 3

In [8]:
def fixi(f):
    x = 1
    res = []
    for i in range(0,10):
        x = f(x)
        res.append(x)
        
    return res

### a) $ x -> 0.5x +  \frac{1}{x}$

In [9]:
f = lambda x: 0.5*x + 1/x
f = fixi(f)

### b) $ x -> \frac{2}{3}x +  \frac{2}{3x}$

In [10]:
g = lambda x: 2/3*x + 2/(3*x) 
g = fixi(g)

### c) $ x -> \frac{3}{4}x +  \frac{3}{4x}$

In [11]:
h = lambda x: 3/4*x + 3/(4*x)
h = fixi(h)

In [12]:
data = {'f': f,'g':g, 'h':h}
df= pd.DataFrame(data)
pd.set_option("display.precision", 16)

print("r =", pow(2, 0.5), end="\n\n")
print(df)

r = 1.4142135623730951

                    f                   g                   h
0  1.5000000000000000  1.3333333333333333  1.5000000000000000
1  1.4166666666666665  1.3888888888888888  1.6250000000000000
2  1.4142156862745097  1.4059259259259260  1.6802884615384617
3  1.4142135623746899  1.4114673015129635  1.7065682774843183
4  1.4142135623730949  1.4132999231994867  1.7194046690076297
5  1.4142135623730949  1.4139092128583717  1.7257509912233235
6  1.4142135623730949  1.4141121343723302  1.7289066487316345
7  1.4142135623730949  1.4141797554645006  1.7304801576280711
8  1.4142135623730949  1.4142022936729557  1.7312658389939559
9  1.4142135623730949  1.4142098061696458  1.7316584122590377


#### from fastest to slowest: (f), (g), (h)
#### where 
#### f :  $ x -> 0.5x +  \frac{1}{x}$
#### g: $ x -> \frac{2}{3}x +  \frac{2}{3x}$
#### h: $ x -> \frac{3}{4}x +  \frac{3}{4x}$

----

## Problem 4

In [13]:
def Newton(f, df, x):
    return x - (f(x)/df(x))

def correct_digits(rel_err):
    return -int(np.log(2*rel_err))

### a) Newton's method

In [14]:
z = 2*np.log(2)
f = lambda x: x*exp(x) - z
df = lambda x: exp(x) + x*exp(x)

c = Newton(f,df,z)
rel_err = abs(c - np.log(2))

c_val = [c]
rel_error = [rel_err]

for i in range(1,6):
    c = Newton(f,df,c)
    rel_err = abs(c - np.log(2))
    
    c_val.append(c)
    rel_error.append(rel_err)
    

In [15]:
data = {"x new":c_val, "rel error": rel_error}
df= pd.DataFrame(data)
pd.set_option("display.precision", 16)

print(df)

                x new           rel error
0  0.9505891992671945  0.2574420187072493
1  0.7379518598316628  0.0448046792717175
2  0.6946966574945903  0.0015494769346450
3  0.6931490879988578  0.0000019074389125
4  0.6931471805628389  0.0000000000028936
5  0.6931471805599453  0.0000000000000000


#### At 4 iterations are required to get atleast 10 digit accuracy (16 digits)

#### Quadratic convergence

### b)  Halley's method

In [16]:
def Halley(f, df, ddf, x):
    return x - ((2*f(x)*df(x))/((2*pow(df(x),2)) - (f(x)*ddf(x))))

In [17]:
z = 2*np.log(2)
f = lambda x: x*exp(x) - z
df = lambda x: exp(x) + x*exp(x)
ddf = lambda x: 2*exp(x) + x*exp(x)

c = Halley(f,df,ddf,z)

rel_err = abs(c - np.log(2))

c_val = [c]
rel_error = [rel_err]

for i in range(1,3):
    c = Halley(f,df,ddf,c)
    rel_err = abs(c - np.log(2))
    
    c_val.append(c)
    rel_error.append(rel_err)

In [18]:
data = {"x new":c_val, "rel error": rel_error}
df= pd.DataFrame(data)
pd.set_option("display.precision", 16)

print(df)

                x new           rel error
0  0.7556183301176645  0.0624711495577193
1  0.6932109077151907  0.0000637271552454
2  0.6931471805600149  0.0000000000000696


---

## Problem 5

### a) Newton vd Waal

In [19]:
R = 0.0820578
T = 320
P = 15
n = 1

V0 = n*R*T/P  

print("init guess =", V0)

init guess = 1.7505664


In [20]:
a = 1.36
b = 0.003183

f = lambda v: (P + pow(n,2)*a/pow(v,2))*(v - n* b) - n*R*T
df = lambda v: P - pow(n,2)*a/pow(v,2) + 2*pow(n,3)*a*b/pow(v,3)

c = V0
c_val = [V0]

tol = True
expected = fsolve(f, x0=V0, xtol=pow(10,-10))[0]

rel_error = [abs(expected-V0)]

while tol:
    c_new = Newton(f,df, c)
    
    tol = abs(c - c_new) > pow(10,-10)
    
    if not tol:
        break
        
    c_val.append(c_new)
    rel_error.append(abs(expected-c_new))
    
    c = c_new
    


In [21]:
print("Volume from fsolve:", expected)

Volume from fsolve: 1.700532565663103


In [22]:
data = {'Volume (Newton)': c_val, 'Error':rel_error}

df= pd.DataFrame(data)
pd.set_option("display.precision", 16)

print(df)

      Volume (Newton)               Error
0  1.7505664000000001  0.0500338343368971
1  1.7005771958987335  0.0000446302356305
2  1.7005325657007959  0.0000000000376930


### b) Using secant

In [23]:
def secant(f, x0, x1):
    return x1, x1 - (f(x1)*(x1-x0)/(f(x1)-f(x0)))

In [24]:
f = lambda v: (P + pow(n,2)*a/pow(v,2))*(v - n* b) - n*R*T

x0, x1 = secant(f, 0.9*V0, V0)
c_val = [0.9*V0,x0,x1]
error = []
Tol = True
while Tol:
    x0, x1 = secant(f, x0, x1)
    
    
    
    Tol = abs(x0 - x1) > pow(10,-10)
    
    if not Tol:
        break
        
    c_val.append(x1)
    
for x in c_val:
    error.append(abs(x-expected))

In [25]:
data = {'Volume (Secant)': c_val, 'Error':error}

df= pd.DataFrame(data)
pd.set_option("display.precision", 16)

print(df)

      Volume (Secant)               Error
0  1.5755097600000001  0.1250228056631029
1  1.7505664000000001  0.0500338343368971
2  1.7004082605234034  0.0001243051396995
3  1.7005324514160529  0.0000001142470500
4  1.7005325656633716  0.0000000000002687


#### Secant (5 iters.) converges slower than Newton (4 iters.) to reach tolerance.

### c.i) Inverse Quadratic Interpolation

In [26]:
def IQI(f, a, b, c):
    
    a_frac = f(b)*f(c)/((f(a) - f(b)) * (f(a) - f(c)))
    a_frac *= a
    
    b_frac = f(a)*f(c)/((f(b) - f(a)) * (f(b) - f(c)))
    b_frac *= b
    
    c_frac = f(a)*f(b)/((f(c) - f(a)) * (f(c) - f(b)))
    c_frac *= c
    
    return b, c ,a_frac + b_frac + c_frac

In [27]:
f = lambda v: (P + pow(n,2)*a/pow(v,2))*(v - n* b) - n*R*T

x1, x2 = secant(f, 0.9*V0, V0)
x0 = x1*0.9

c_val = [x0,x1,x2]
error = []
Tol = True

In [28]:
while Tol:
    x0, x1, x2 = IQI(f, x0, x1, x2)
    Tol = abs(x1 - x2) > pow(10,-10)
    if not Tol:
        break
    
    c_val.append(x2)
    
for x in c_val:
    error.append(abs(x-expected))
    
data = {'Volume (IQI)': c_val, 'Error':error}

df= pd.DataFrame(data)
pd.set_option("display.precision", 16)

print(df)

         Volume (IQI)               Error
0  1.5755097600000001  0.1250228056631029
1  1.7505664000000001  0.0500338343368971
2  1.7004082605234034  0.0001243051396995
3  1.7005325753333793  0.0000000096702764
4  1.7005325656631018  0.0000000000000011


#### IQI (5 iters.) converges slower than Newton (4 iters.) to reach tolerance.

### c.ii) Steffensen

In [29]:
def steffman(f, x0):
    res = [x0]
    Tol = True
    while Tol:
        
        x = x0 - pow(f(x0), 2)/(f(x0 + f(x0)) - f(x0))
        
        Tol = abs(x-x0) > pow(10,-10)
        
        if not Tol:
            break
            
        res.append(x)
        
        x0 = x

    return res

In [30]:
f = lambda v: (P + pow(n,2)*a/pow(v,2))*(v - n* b) - n*R*T

c_val = steffman(f, V0)

In [31]:
error = []
for x in c_val:
    error.append(abs(x-expected))
    
data = {'Volume (Steffensen)': c_val, 'Error':error}

df= pd.DataFrame(data)
pd.set_option("display.precision", 16)

print(df)

   Volume (Steffensen)               Error
0   1.7505664000000001  0.0500338343368971
1   1.7010185492929391  0.0004859836298361
2   1.7005326347473626  0.0000000690842596
3   1.7005325656631043  0.0000000000000013


#### Steffensen's (4 iters.) converges slightly slower Newton (4 iters.) and slightly less accurate (at i = 3) to reach tolerance.

### d) Algebra

#### $ V^2 (P + \frac{n^2 a}{V^2}) (V - nb) = V^2 nRT$
#### $ (PV^2 + n^2a) (V - nb) = V^2 nRT$ 
#### $ PV^3 - nPV^2b +an^2V - an^3b = V^2 nRT$
#### $ (P)V^3 - (nPb + nRT)V^2 +(an^2)V - (an^3b) = 0 $

In [32]:
R = 0.0820578
T = 320
P = 15
n = 1

a = 1.36
b = 0.003183

coeff = [P, -(n*P*b + n*R*T), a*pow(n,2), -a*pow(n,3)*b]
roots = np.roots(coeff)

print("V found: ", roots)

V found:  [1.70053257 0.04980973 0.0034071 ]


In [33]:
print("Using numpy roots function: V = {} L".format(roots[0]))
print("error: {:.16f}".format(abs(expected - roots[0])))

Using numpy roots function: V = 1.7005325656631043 L
error: 0.0000000000000013


---

## Problem 6

### let $   x = P_{\infty}$

In [34]:
def generateJ(x,c,k):
    J00 = 1/(c+1)
    J01 = -x/(c+1)**2
    J02 = 0

    J10 = 1/(c*exp(-k) +1)
    J11 = -exp(-k)*x/(c*exp(-k) +1)**2
    J12 = c*exp(-k)*x/ (c*exp(-k) +1)**2

    J20 = 1/(c*exp(-2*k)+1)
    J21 = -exp(-2*k)*x/(c*exp(-2*k) +1)**2
    J22 = 2*c*exp(-2*k)*x/(c*exp(-2*k) +1)**2

    J = [[J00, J01, J02],
         [J10, J11, J12],
         [J20, J21, J22]
        ]
    
    return J

In [35]:
def generatefx(x,c,k):
    f1 = x/(1+c) - 1.32
    f2 = x/(1+c * exp(-k)) - 1.79
    f3 = x/(1+c*exp(-k*2)) - 2.26
    
    fx = [[f1], [f2], [f3]]
    
    return fx

In [36]:
def unpack(X):
    return X[0][0], X[1][0], X[2][0]

In [37]:
X = [[4], [2], [0.5]]
it = 1

res = {}
res['k'] = []
res['P_inf'] = []
res['c'] = []

while it < 6:
    x, c, k = unpack(X)

    f_x = generatefx(x,c,k)
    J = generateJ(x,c,k)
    J_inv = np.linalg.inv(J)

    X = X - np.dot(J_inv, f_x)
    
    res['k'].append(abs(k - np.log(113/66)))
    res['P_inf'].append(abs(x - 3.58))
    res['c'].append(abs(c - 113/66))
    

    it += 1
    
print("solution:\n\nP_inf = {}\nc = \t{}\nk=\t{}".format(x,c,k))
print("expected:\n\nP_inf = {}\nc = \t{}\nk=\t{}".format(3.58, 113/66, np.log(113/66)))

solution:

P_inf = 3.5799999999999863
c = 	1.7121212121212017
k=	0.5377330766859163
expected:

P_inf = 3.58
c = 	1.7121212121212122
k=	0.5377330766859151


In [38]:
df = pd.DataFrame(res)
print(df)

                    k               P_inf                   c
0  0.0377330766859151  0.4199999999999999  0.2878787878787878
1  0.0013569777833489  0.0608278840444338  0.0427421251545377
2  0.0000858653507353  0.0002017307742839  0.0002065585575337
3  0.0000000063279778  0.0000001125872742  0.0000000893871619
4  0.0000000000000012  0.0000000000000138  0.0000000000000104


#### At each iteration of newton's method for systems, the error in the values computed for (k, P_inf and c) diminishes quadratically. 

## Problem 7

In [39]:
a = 0.138*pow(10, -6)
t = 5184000 
T_i = 20
T_s = -15

# reorder s.t T(x,t) = ... formula
T = lambda x: erf(x*1/(2*pow(a*t, 0.5)))*(T_i - T_s) + T_s

#built in solver
x1 = fsolve(T, x0=0, xtol=0.00000001)[0]
print("{} meters".format(round(x1,5)))

0.67696 meters


---

# FIN